# Jacobi's Method

In [1]:
import numpy as np
import numpy.linalg as npla

# New additions!
import scipy
from scipy import sparse

In [2]:
# Let's do a simple Ax = b problem with a 3x3 matrix A
# Normally, you'd employ a MUCH larger matrix with Jacobi's Method...

A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A =\n", A, "\n\nb =", b)

# What's the ACTUAL (ideal) solution for x (not iteration, just straight-out solution)??
xideal = npla.solve(A,b)
print("\nIf Ax = b, then x = ", xideal)

A =
 [[ 4 -1 -1]
 [-2  6  1]
 [-1  1  7]] 

b = [ 3  9 -6]

If Ax = b, then x =  [ 1.  2. -1.]


### Using Jacobi's Method - the Matrix view:

*What do you need to start off with? See this:*

In [3]:
# Get dimensions of matrix A:
m, n = A.shape

# Get the diagonals as a vector d:
d = A.diagonal()

# Convert that diagonals vector d into a diagonal MATRIX D:
D = np.diag(d)

print("\nm = ", m, ";","n = ", n, "\n")
print("d =\n", d, "\n")
print("D =\n", D, "\n")

# Create matrix C, which is A WITHOUT the diagonals
C = A - D
print("C =\n", C, "\n")

# Let's make an initial guess: x = 0
x = np.zeros(n)
print ("inital guess for x, i.e. x[0] = ", x)


m =  3 ; n =  3 

d =
 [4 6 7] 

D =
 [[4 0 0]
 [0 6 0]
 [0 0 7]] 

C =
 [[ 0 -1 -1]
 [-2  0  1]
 [-1  1  0]] 

inital guess for x, i.e. x[0] =  [0. 0. 0.]


We KNOW (this is like "cheating" b/c we ran `npla.solve()`) that `x =  [1, 2, -1]`

***So let's improve on the initial guess of x = [0,0,0]:***

In [4]:
xnew = (b - C @ x) / d
print( "x[1] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)

# residual = xnew - xideal
# error = npla.norm( xnew - xideal )
# relres = npla.norm( xnew - xideal ) / npla.norm( xideal )
# print( "residual:", residual, "\nerror:", error, "\nrelres:", relres )

x[1] =  [ 0.75        1.5        -0.85714286]
error: 2.5557558089735486


***Ok - better, but not close enough (relative residual is too high). Do it again!***

In [5]:
xnew = (b - C @ xnew) / d
print( "x[2] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)


x[2] =  [ 0.91071429  1.89285714 -0.96428571]
error: 0.5649747149841555


***Ok - AGAIN, it's better, but not close enough (relative residual is too high). Do it again!***

In [6]:
xnew = (b - C @ xnew) / d
print( "x[3] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)


x[3] =  [ 0.98214286  1.96428571 -0.99744898]
error: 0.18013166779499973


### Ok - you see where this is going? Better do a loop!

In [7]:
#Again, start with our initial guess of [0,0,0]:
x = np.zeros(3)

for i in range( 100 ):
    x = (b - C @ x) / d
    error = npla.norm( A@x - b)
    print( "iteration", i + 1, "x:", x, ", error:" ,error )
    if error <= 1e-8:
        break

iteration 1 x: [ 0.75        1.5        -0.85714286] , error: 2.5557558089735486
iteration 2 x: [ 0.91071429  1.89285714 -0.96428571] , error: 0.5649747149841555
iteration 3 x: [ 0.98214286  1.96428571 -0.99744898] , error: 0.18013166779499973
iteration 4 x: [ 0.99170918  1.99362245 -0.99744898] , error: 0.04021904343221404
iteration 5 x: [ 0.99904337  1.99681122 -1.00027332] , error: 0.01798088346639159
iteration 6 x: [ 0.99913448  1.99972668 -0.99968112] , error: 0.004522016940133618
iteration 7 x: [ 1.00001139  1.99965835 -1.0000846 ] , error: 0.0024020931567846297
iteration 8 x: [ 0.99989344  2.0000179  -0.99994957] , error: 0.0007811656507351477
iteration 9 x: [ 1.00001708  1.99995607 -1.00001778] , error: 0.00038839902247926573
iteration 10 x: [ 0.99998457  2.00000866 -0.99999128] , error: 0.0001478837545685464
iteration 11 x: [ 1.00000434  1.99999341 -1.00000344] , error: 6.819334618850008e-05
iteration 12 x: [ 0.99999749  2.00000202 -0.99999844] , error: 2.7833883358805923e-05


### We see from the results above that if we (arbitrarily) chose a threshold of 1e-8, that iteration number 19 would get us just below that!!

## ***BUT!!!*** Jacobi's Method does not always converge...! :(

In [8]:
# Example that does NOT converge using J. Method:

A = np.array([[1,2],[3,4]])
b = np.array([3,7])
print("A:\n", A)
print("\nb = ", b)
x = npla.solve(A, b)
print("\nx (ideal) = ", x)

A:
 [[1 2]
 [3 4]]

b =  [3 7]

x (ideal) =  [1. 1.]


In [9]:
# Get d, D, and C:
d = A.diagonal()
D = np.diag(d)
C = A - D

#Start with our initial guess of [0,0]:
x = np.zeros(2)

for i in range( 100 ):
    x = (b - C @ x) / d
    error = npla.norm( A@x - b)
    print( "iteration", i + 1, "x:", x, ", error:" ,error )
    if error <= 1e-8:
        break

iteration 1 x: [3.   1.75] , error: 9.656603957913983
iteration 2 x: [-0.5 -0.5] , error: 11.423659658795863
iteration 3 x: [4.    2.125] , error: 14.484905936870975
iteration 4 x: [-1.25 -1.25] , error: 17.135489488193794
iteration 5 x: [5.5    2.6875] , error: 21.727358905306463
iteration 6 x: [-2.375 -2.375] , error: 25.70323423229069
iteration 7 x: [7.75    3.53125] , error: 32.59103835795969
iteration 8 x: [-4.0625 -4.0625] , error: 38.55485134843604
iteration 9 x: [11.125     4.796875] , error: 48.88655753693954
iteration 10 x: [-6.59375 -6.59375] , error: 57.832277022654054
iteration 11 x: [16.1875     6.6953125] , error: 73.3298363054093
iteration 12 x: [-10.390625 -10.390625] , error: 86.74841553398107
iteration 13 x: [23.78125     9.54296875] , error: 109.99475445811396
iteration 14 x: [-16.0859375 -16.0859375] , error: 130.1226233009716
iteration 15 x: [35.171875   13.81445312] , error: 164.99213168717094
iteration 16 x: [-24.62890625 -24.62890625] , error: 195.1839349514574

## We see from the results above that we NEVER CONVERGE!!

#### We could have avoided this "heartache" by checking the "Spectral Radius" of the Matrix A:

In [10]:
# Check spectral radius
m = npla.inv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

[ 1.22474487 -1.22474487]
Spectral radius >= 1. Will not converge.


In [11]:
# Check it again for our earlier matrix A (that DID converge)
# We'll call it matrix Z here, just to distinguish it from matrix A above:

Z = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
d = Z.diagonal()
D = np.diag(d)
C = Z - D

# Check spectral radius
m = npla.pinv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

[ 0.42946179 -0.28202928 -0.14743251]
Spectral radius < 1. Will converge.


### Let's create a function that can do all of this for us!

**Presenting function `Jsolve()`:** \
**It takes in our matrix `A`, vector `b`and gives us the best solution for `x` (plus the `resrel`)** \

It should also have as arguments: a threshold tolerance (default = 1e-8), maximum number of iterations (default = 1000)

# ATTENTION:

**`Jsolve()` employs SPARSE MATRICES so that it's use can be extended to very large, sparse matrices, as well as, more "ordinary" ones.**

This means that BEFORE using it, make sure to convert an np.array() type matrix into a sparse one (how to do that is illustrated all the way below):

In [18]:
def Jsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the Jacobi iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration. 
        The callback function has arguments 'x', 'iteration', and 'residual'
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Split A into diagonal D plus off-diagonal C
    d = A.diagonal()         # diagonal elements of A as a vector
    C = A.copy()             # copy of A ...
    C.setdiag(np.zeros(n))   # ... without the diagonal
    
    # Initial guess: x = 0
    x = np.zeros(n)

    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
        
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range (max_iters):
        # New x
        x = (b - C @ x) / d

        # Record relative residual (this can be done instead of error)
        # Remember: rel_res = error / some_relative_reference
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k+1, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [19]:
A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A:\n", A)
print("\nb:\n", b)
x = npla.solve(A, b)
print("\nIdeal x (so we can compare against it):\n", x)

A:
 [[ 4 -1 -1]
 [-2  6  1]
 [-1  1  7]]

b:
 [ 3  9 -6]

Ideal x (so we can compare against it):
 [ 1.  2. -1.]


In [20]:
#Run it using Jacobi - note, Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)

print("x: \n", Jsolve(A, b)[0])
print("\nAll iterated residuals: \n", Jsolve(A, b)[1])

# To see just the last residual:
# NOTE: [1] indicates element 1 of the function return, which is a list,
#       [-1] indicates the LAST element in that list.

print("\nLast residual: ", Jsolve(A,b)[1][-1])

x: 
 [ 1.00000001  1.99999999 -1.        ]

All iterated residuals: 
 [1.0, 0.22768482384372116, 0.0503319479919278, 0.01604740441421596, 0.003582997165407753, 0.001601864415303465, 0.00040285328778971063, 0.00021399546675719867, 6.959176731826758e-05, 3.4601335035126733e-05, 1.3174531968220623e-05, 6.075146130834141e-06, 2.479639411240011e-06, 1.099349933223233e-06, 4.6202066343380954e-07, 2.0119957277470704e-07, 8.561022957943247e-08, 3.698871997266093e-08, 1.5822059686393463e-08, 6.8127044622987766e-09]

Last residual:  6.8127044622987766e-09


In [21]:
A = np.array([[4, 0, 1, 0], [2, 7, 7, 2], [1, 1, 4, 4], [0, 0, 2, 6]])
b = np.array([1, 2, 3, 4])
print(npla.solve(A,b))

[ 0.2406015  -0.0075188   0.03759398  0.65413534]


In [22]:
d = A.diagonal()
D = np.diag(d)
C = A - D

# Check spectral radius
m = npla.pinv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

[ 8.34137362e-01 -7.69197180e-01 -6.49401819e-02  2.64091342e-18]
Spectral radius < 1. Will converge.


In [23]:
#Run it using Jacobi - note, Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)
solution = Jsolve(A, b, tol = 1e-8)
print("x: \n",solution[0])
print("\nAll iterated residuals: \n", solution[1])
print("length = ", len(solution[1]))
print("\nLast residual: ", solution[1][-1])

x: 
 [ 0.2406015  -0.00751879  0.03759399  0.65413534]

All iterated residuals: 
 [1.0, 1.4519108658968871, 1.2912751537070846, 0.9865560160549319, 0.8941390714131437, 0.69069042722823, 0.6183939725979902, 0.48319316304257875, 0.42806850186917, 0.3377617814670239, 0.29654865875975306, 0.23594048162245626, 0.20557155530601262, 0.16471760979850092, 0.14258428350857297, 0.11493716890519119, 0.09894310115933581, 0.08016703233839535, 0.0686868697808074, 0.05589504464393763, 0.047699085565958525, 0.03895976787655909, 0.03313387518080819, 0.027148442047978002, 0.02302190971046195, 0.018913677294891663, 0.01599931290166035, 0.013174193953853138, 0.011120864358881427, 0.009174903498598782, 0.007731101757595503, 0.006388793602497441, 0.005375266754515825, 0.004448207884024024, 0.003737714042181043, 0.0030967617118022157, 0.002599276204842514, 0.002155725668118326, 0.0018077278786830806, 0.001500540478481794, 0.001257311276884608, 0.0010444199521136288, 0.0008745352740619085, 0.000726908634724655